# Deploy a Hyper-Segmented Model with TDStone2 in less than a minute
# Using Scikit Learn Regressor Pipeline

## 1 - Connect to Vantage

In [1]:
import teradataml as tdml
import json
import warnings
warnings.filterwarnings('ignore')
tdml.__version__

'20.00.00.03'

In [2]:
with open('../connections/me.json','r') as f:
    Param_ = json.load(f)

with open('../connections/csae2.json','r') as f:
    Param = json.load(f)

with open('../connections/csae.json','r') as f:
    Param_ = json.load(f)

with open('../connections/vantage24.json','r') as f:
    Param_ = json.load(f)

tdml.create_context(**Param)

Engine(teradatasql://:***@sto-xo0saiv0ot1sxmj2.env.clearscape.teradata.com?DATABASE=demo_user&user=DEMO_USER)

## 2 - The Hyper-Segmented Dataset

In [3]:
dataset = tdml.DataFrame(tdml.in_schema(Param['database'],'dataset_00'))
dataset

Partition_ID,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,flag,Y1,Y2,FOLD
9,10010,0.2958804484987114,0.003716488369000244,0.01722984108468596,-0.8980370254646806,-0.7133844217285864,1.4033073897256012,0.09585056813091393,-1.5727605216032974,0.24669220433822078,0,0.30614073239501466,1,train
9,10013,0.21218527816317997,2.166812356509659,-0.1266757732207366,1.9474973269441456,0.13917028091362654,0.20430437102374058,-1.3606770865820423,-0.05685071859701827,2.291005491276908,0,-0.9461899572379499,0,train
9,10014,0.2696428584641046,0.09665053813760335,-1.0018103382973418,0.2303663593320063,-1.4484362655020413,0.3048885822345157,0.5428096467932015,2.495707581732601,-1.9600913126996844,0,-0.7660544464255463,0,train
9,10015,1.0352969567784633,1.7180803172505401,-1.212333026542816,0.23062189483724332,0.09487671261837792,1.4785411000201696,-0.1915928514243548,0.2738925677755579,0.4567414265556841,0,1.1925536062692985,1,train
9,10018,-0.056727019348374944,0.10544771580930327,1.8454204127385498,0.3616908074784603,1.2205212060733945,-0.06193245371322754,-0.5530192900801478,-0.43005691181938516,-0.4488047777379936,1,-1.837775730009315,0,train
9,10019,0.9477234671587255,-1.0709365555193369,0.4104635207616072,-0.19428923876533416,0.9744406394994841,-1.595214852414057,1.7279071952164111,-0.1411204037244582,0.8182704738164543,0,1.9734907309554794,1,train
9,10017,-2.0955851487485213,-0.12377353233081276,0.753201306001975,-0.5383166736630877,-0.6259345787552749,0.08287535513834084,0.8284104984331996,0.2097047767930522,-0.4857437685427433,0,-0.07178812944773284,0,train
9,10011,-0.21262362685570874,-1.6603317529582815,1.7388731099892527,-1.5923116289115609,0.26452601206465765,-0.46762101872188044,-1.0733257888941525,-0.6245670487544273,0.24821063575400967,0,2.183759325597856,1,train
9,10009,-0.19855839525509705,1.5590910463488572,-0.24941730720279,1.3439589643048995,-0.2727814812771797,-1.3409329048433465,-1.855851730498525,-0.6037874835632588,-1.1436998323386687,0,1.6793888752832822,1,train
9,10006,0.8715036971507598,-0.9547011166837505,-0.7212371657720575,-0.513759234558512,0.16062548990127876,-0.09512967554627166,-0.35991104923966133,-0.8666125648617071,-0.764595423910862,1,-0.3168103113201745,0,train


In [4]:
summary = dataset.groupby('Partition_ID').count()
summary

Partition_ID,count_ID,count_X1,count_X2,count_X3,count_X4,count_X5,count_X6,count_X7,count_X8,count_X9,count_flag,count_Y1,count_Y2,count_FOLD
9,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
6,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
1,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
4,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000


In [5]:
summary.shape

(4, 15)

## 3 - Hyper-segmented model deployment

### 3.1 - Engineering of the scikit-learn classifier pipeline

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
# Example usage
steps_classifier = [
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(
        max_depth = 5,
        n_estimators = 95 
    ))]

### 3.2 - Deployment of the scikit-learn pipeline

In [7]:
from tdstone2.tdshypermodel import HyperModel
from tdstone2.tdstone import TDStone
sto = TDStone(schema_name = Param['database'], SEARCHUIFDBPATH = Param['user'])

In [ ]:
model_parameters = {
    "target": 'Y1',
    "column_categorical": ['flag'],
    "column_names_X": ['X1','X2','X3','X4','X5','X6','X7','X8','X9','flag']
}

In [9]:
%%time
model = HyperModel(tdstone            = sto,
                   metadata           = {'project': 'test'},
                   skl_pipeline_steps = steps_classifier,
                   model_parameters   = model_parameters,
                   dataset            = tdml.in_schema(Param['database'],'dataset_00'),
                   id_row             = 'ID',
                   id_partition       = 'Partition_ID',
                   id_fold            = 'FOLD',
                   fold_training      = 'train')

registration of mapper with id = dc1cdd3e-111f-440e-ae0e-c0b3336c0716
creation of dedicated mapper table : demo_user.TDS_MAPPER_dc1cdd3e_111f_440e_ae0e_c0b3336c0716
creation of the on clause view demo_user.TDS_ON_CLAUSE_TRAINING_dc1cdd3e_111f_440e_ae0e_c0b3336c0716
creation of the sto view demo_user.TDS_STO_TRAINING_dc1cdd3e_111f_440e_ae0e_c0b3336c0716
registration of mapper with id = 431b1e0c-57a4-4ae8-bb3e-3a2372349857
creation of dedicated mapper table : demo_user.TDS_MAPPER_431b1e0c_57a4_4ae8_bb3e_3a2372349857
creation of the on clause view demo_user.TDS_ON_CLAUSE_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857
creation of the volatile table on on clause view demo_user.TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857
creation of the sto view demo_user.TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857
register hyper model with id : c5ef0032-dd20-4b48-817e-6fe7e3c42e55
hyper model : c5ef0032-dd20-4b48-817e-6fe7e3c42e55
CPU times: total: 188 ms
Wall time: 27.6 s


In [10]:
sto.list_hyper_models()

CREATION_DATE,ID,ID_MODEL,ID_MAPPER_TRAINING,ID_MAPPER_SCORING,METADATA
2025-01-08 05:31:04.970000-05:,c5ef0032-dd20-4b48-817e-6fe7e3c42e55,505ea202-bbf4-4b2c-9639-2a18176d97f1,dc1cdd3e-111f-440e-ae0e-c0b3336c0716,431b1e0c-57a4-4ae8-bb3e-3a2372349857,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:26:48.160000-05:,654b028b-741e-4a46-8e5d-d966ac8a07b1,1feb2a86-622c-4c56-8e94-4c367d1a232e,05842693-ddf1-4589-8d7b-a7ea4e952b95,a93604aa-2204-45a6-b4f5-1edab0414da1,"{""user"": ""dm250067"", ""project"": ""test""}"


In [11]:
id = sto.list_hyper_models()[['ID','CREATION_DATE']].to_pandas().reset_index().sort_values('CREATION_DATE', ascending=False).iloc[0,0]
id

'c5ef0032-dd20-4b48-817e-6fe7e3c42e55'

## 4 - Execution of the deployed hypermodel

### 4.1 - Models Training

In [12]:
%%time
model.train()

insert trained models in demo_user.TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716
- access pickle models in demo_user.V_TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716_PICKLE
- access  onnx  models in demo_user.V_TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716_ONNX
- onnx byom catalog    in demo_user.V_TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716_BYOM_CATALOG
CPU times: total: 0 ns
Wall time: 11.7 s


In [13]:
model.get_trained_models().groupby('TD_TIMECODE').count()

demo_user TDS_MODEL_REPOSITORY


TD_TIMECODE,count_ID_PROCESS,count_Partition_ID,count_ID_MODEL,count_ID_TRAINED_MODEL,count_MODEL_TYPE,count_STATUS
9999-01-01 00:00:00.000000-04:,4,4,4,4,4,4


### 4.2 - Model Scoring

In [14]:
%%time
model.score()

creation of the on clause view demo_user.TDS_ON_CLAUSE_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857
creation of the volatile table on on clause view demo_user.TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857
insert scores in demo_user.TDS_SCORES_431b1e0c_57a4_4ae8_bb3e_3a2372349857
CPU times: total: 0 ns
Wall time: 16.4 s


In [15]:
model.get_model_predictions().groupby('TD_TIMECODE').count()

demo_user TDS_SCORES_431b1e0c_57a4_4ae8_bb3e_3a2372349857


TD_TIMECODE,count_ID_PROCESS,count_ID_TRAINED_MODEL,count_Partition_ID,count_ID,count_Y1_prediction
9999-01-01 00:00:00.000000-04:,40000,40000,40000,40000,40000


In [16]:
model.get_model_predictions()

demo_user TDS_SCORES_431b1e0c_57a4_4ae8_bb3e_3a2372349857


TD_TIMECODE,ID_PROCESS,ID_TRAINED_MODEL,Partition_ID,ID,Y1_prediction
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,5cc8f430-6299-453e-81e5-84129c6d0a83,4,28742,0.0395799030283947
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,de9b490e-a88e-45d7-b825-243ed9ebdc21,6,36986,1.7912688347819974
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,5cc8f430-6299-453e-81e5-84129c6d0a83,4,20158,0.06744816554339858
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,5cc8f430-6299-453e-81e5-84129c6d0a83,4,27596,0.00294134516987689
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,de9b490e-a88e-45d7-b825-243ed9ebdc21,6,36073,0.02698896376185059
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,a3071898-3304-430c-8576-cf5b00e8c610,9,18306,0.03793123140570147
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,de9b490e-a88e-45d7-b825-243ed9ebdc21,6,32307,0.0327810051781828
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,5cc8f430-6299-453e-81e5-84129c6d0a83,4,23877,-0.04959107234946412
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,a3071898-3304-430c-8576-cf5b00e8c610,9,13503,-0.0765072842653416
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,16ddf980-c03c-4fcc-936c-0ba7ca631548,1,7828,0.07985155274176071


## 5 - A bit of administration: Model Lineage

### 5.1 - Access to the list of deployed codes

In [17]:
sto.list_codes()

ID,CODE_TYPE,METADATA
c88da75f-4c35-4fb4-8b2b-52105610adf7,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"
0a923c33-9369-407e-80a0-187ffa2a9837,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"


### 5.2 - List of deployed models (code + parameters)

In [18]:
sto.list_models()

ID,ID_CODE,ARGUMENTS,METADATA
505ea202-bbf4-4b2c-9639-2a18176d97f1,0a923c33-9369-407e-80a0-187ffa2a9837,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""target"": ""Y1"", ""column_categorical"": [""flag""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""arguments"": {""regressor__max_depth"": 5, ""regressor__n_estimators"": 95}}}","{""user"": ""dm250067"", ""project"": ""test""}"
1feb2a86-622c-4c56-8e94-4c367d1a232e,c88da75f-4c35-4fb4-8b2b-52105610adf7,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""target"": ""Y2"", ""column_categorical"": [""flag"", ""Y2""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""arguments"": {""classifier__max_depth"": 5, ""classifier__n_estimators"": 95}}}","{""user"": ""dm250067"", ""project"": ""test""}"


### 5.3 - List of available mappers (mapping between partitions and models or trained models)

In [19]:
sto.list_mappers()

ID,MAPPER_TYPE,TABLE_NAME,CODE_REPOSITORY,MODEL_REPOSITORY,TRAINED_MODEL_REPOSITORY,FEATURE_REPOSITORY,SCORES_REPOSITORY,DATASET_OBJECT,COL_ID_ROW,COL_ID_PARTITION,COL_FOLD,ON_CLAUSE_VIEW,STO_VIEW,METADATA
05842693-ddf1-4589-8d7b-a7ea4e952b95,training,TDS_MAPPER_05842693_ddf1_4589_8d7b_a7ea4e952b95,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_05842693_ddf1_4589_8d7b_a7ea4e952b95,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_05842693_ddf1_4589_8d7b_a7ea4e952b95,TDS_STO_TRAINING_05842693_ddf1_4589_8d7b_a7ea4e952b95,"{""user"": ""dm250067""}"
431b1e0c-57a4-4ae8-bb3e-3a2372349857,scoring,TDS_MAPPER_431b1e0c_57a4_4ae8_bb3e_3a2372349857,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,None,TDS_SCORES_431b1e0c_57a4_4ae8_bb3e_3a2372349857,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,"{""user"": ""dm250067""}"
dc1cdd3e-111f-440e-ae0e-c0b3336c0716,training,TDS_MAPPER_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,TDS_STO_TRAINING_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,"{""user"": ""dm250067""}"
a93604aa-2204-45a6-b4f5-1edab0414da1,scoring,TDS_MAPPER_a93604aa_2204_45a6_b4f5_1edab0414da1,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_05842693_ddf1_4589_8d7b_a7ea4e952b95,None,TDS_SCORES_a93604aa_2204_45a6_b4f5_1edab0414da1,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_a93604aa_2204_45a6_b4f5_1edab0414da1,TDS_STO_SCORING_a93604aa_2204_45a6_b4f5_1edab0414da1,"{""user"": ""dm250067""}"


### 5.4 - List of Hypermodels ( models and mappers mapping)

In [20]:
sto.list_hyper_models()

CREATION_DATE,ID,ID_MODEL,ID_MAPPER_TRAINING,ID_MAPPER_SCORING,METADATA
2025-01-08 05:31:04.970000-05:,c5ef0032-dd20-4b48-817e-6fe7e3c42e55,505ea202-bbf4-4b2c-9639-2a18176d97f1,dc1cdd3e-111f-440e-ae0e-c0b3336c0716,431b1e0c-57a4-4ae8-bb3e-3a2372349857,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:26:48.160000-05:,654b028b-741e-4a46-8e5d-d966ac8a07b1,1feb2a86-622c-4c56-8e94-4c367d1a232e,05842693-ddf1-4589-8d7b-a7ea4e952b95,a93604aa-2204-45a6-b4f5-1edab0414da1,"{""user"": ""dm250067"", ""project"": ""test""}"


In [21]:
tdml.remove_context()

True